# Train Model
## Written By KYLiN

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader , random_split
from torchvision.models import mobilenet_v3_large

# speed up 
from torch.cuda.amp import GradScaler , autocast


from rich import print
from tqdm import tqdm
import os
from time import time

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda

In [3]:
# 训练数据的 transforms
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.RandomAffine(degrees=0,translate=(0.05,0.05)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 测试数据的 transforms
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
hot_folder_path , boo_folder_path = "./data/HOT" , "./data/BOO"

In [5]:
hot_data_images = [os.path.join(hot_folder_path , item) for item in os.listdir(hot_folder_path)]
boo_data_images = [os.path.join(boo_folder_path , item) for item in os.listdir(boo_folder_path)]

hot_images_size  , boo_images_size = len(hot_data_images) , len(boo_data_images)

total_images_size = hot_images_size + boo_images_size 

hot_weight , boo_weight = total_images_size / (2*hot_images_size) , total_images_size / (2* boo_images_size)

In [6]:
dataset_path = "./data"
dataset = ImageFolder(dataset_path)

dataset_size = len(dataset)
train_size = int(0.75 * dataset_size)
val_size = dataset_size - train_size

print(f"dataset size: {dataset_size}, train size: {train_size}, val size: {val_size}")

# BOO 0 , HOT 0
sample_weight = torch.Tensor([boo_weight , hot_weight])
print(f"hot weight: {hot_weight} , boo weight: {boo_weight}")


dataset size: 55231, train size: 41423, val size: 13808

hot weight: 2.4789497307001795 , boo weight: 0.6263296364337393

In [7]:
train_dataset , val_dataset = random_split(dataset , [train_size , val_size])

train_dataset.dataset.transform = transform_train
val_dataset.dataset.transform = transform_test


In [8]:
TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

In [9]:
train_loader = DataLoader(train_dataset , batch_size=TRAIN_BATCH_SIZE , shuffle=True)
test_loader = DataLoader(val_dataset , batch_size=TEST_BATCH_SIZE , shuffle=True)

In [10]:
MODEL_PATH = os.path.join("./model" , "mobileNet_v3_test_v2.pth")


model = mobilenet_v3_large()

num_features = model.classifier[-1].in_features
# output only two class 
model.classifier[-1] = nn.Linear(num_features , 2)

model = model.to(device=device)
# print(model)


In [11]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=sample_weight.to(device))
optimizer = optim.Adam(model.parameters() , lr=3e-4 ,  weight_decay=0.0001)

In [12]:
TRAIN_EPOCH = 3

In [15]:
scaler = GradScaler()
old_test_acc = -1
for epoch in range(TRAIN_EPOCH):
    start_time = time()
    
    
    train_acc , test_acc = 0 , 0 
    # training
    model.train()
    
    with tqdm(train_loader , unit="batch" , desc="Training...") as t_epoch:
        for inputs , labels in t_epoch:
            # in cuda
            torch.cuda.empty_cache()
            inputs , labels_gpu = inputs.to(device) , labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            with autocast():
                model_outputs = model(inputs)
                loss = criterion(model_outputs , labels_gpu)
                
            # use scaler update  
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
                
            
            # in cpu
            model_outputs = model_outputs.cpu()
            # dim is one , get array 
            train_pred = torch.max(model_outputs , 1).indices
            # how many is same 
            train_acc += int(torch.sum(train_pred == labels))
        
        # get epoch train acc 
        ep_train_acc = train_acc / train_size
    
    # lock model 
    model.eval()
    with torch.no_grad():
        # validation
        with tqdm(test_loader , unit="batch" , desc="Testing...") as test_epoch:
            for inputs , labels in test_epoch:
                # in cuda
                torch.cuda.empty_cache()
                inputs , labels_gpu = inputs.to(device) , labels.to(device)
                test_prob = model(inputs)
                
                # in cpu
                test_prob = test_prob.cpu()
                test_pred = torch.max(test_prob , 1).indices
                test_acc += int(torch.sum(test_pred == labels))
                
        ep_test_acc = test_acc / val_size
                
    
    end_time = time()
    duration = (end_time - start_time) / 60
    print(f"Time: {duration}, Loss: {loss:.2f}\nTrain_acc: {ep_train_acc*100 :.2f}, Test_acc: {ep_test_acc*100 :.2f}")
    
    if ep_test_acc > old_test_acc:
        torch.save(model.state_dict() , MODEL_PATH)
        old_test_acc = ep_test_acc
        print(f"update new model, new {ep_test_acc*100 :.2f} , save in {MODEL_PATH}")
    
    

Testing...: 100%|██████████| 432/432 [08:02<00:00,  1.12s/batch]


Time: 32.74378747542699, Loss: 0.59
Train_acc: 60.74, Test_acc: 66.43

update new model, new 66.43 , save in ./model\mobileNet_v3_test_v2.pth

Testing...: 100%|██████████| 432/432 [07:22<00:00,  1.02s/batch]


Time: 28.93555169502894, Loss: 0.73
Train_acc: 61.12, Test_acc: 54.69

Testing...: 100%|██████████| 432/432 [07:21<00:00,  1.02s/batch]


Time: 29.043704775969186, Loss: 0.65
Train_acc: 61.35, Test_acc: 71.39

update new model, new 71.39 , save in ./model\mobileNet_v3_test_v2.pth

In [14]:
# save model
# torch.save(model.state_dict() , MODEL_PATH)